In [26]:
import numpy as np
import aicspylibczi
import pathlib
from PIL import Image
from aicspylibczi import CziFile
from pathlib import Path
import matplotlib.pyplot as plt
import os
import czifile
import tifffile
from tqdm import tqdm

mosaic_file = pathlib.Path('czi_files/2023_03_02__0005.czi')
czi = aicspylibczi.CziFile(mosaic_file)


In [27]:
# define helper functions
def norm_by(x, min_, max_):
    norms = np.percentile(x, [min_, max_])
    i2 = np.clip((x - norms[0]) / (norms[1] - norms[0]), 0, 1)
    return i2


def recolor(im):  # transform from rgb to cyan-magenta-yellow
    im_shape = np.array(im.shape)
    color_transform = np.array([[1, 1, 0], [0, 1, 1], [1, 0, 1]]).T
    im_reshape = im.reshape([np.prod(im_shape[0:2]), im_shape[2]]).T
    im_recolored = np.matmul(color_transform.T, im_reshape).T
    im_shape[2] = 3
    im = im_recolored.reshape(im_shape)
    return im

In [28]:
# Get the shape of the data
dimensions = czi.dims  # 'STCZMYX'
dimensions

'SCMYXA'

In [29]:
czi.size  # (1, 1, 1, 1, 2, 624, 924)

(1, 1, 124, 1200, 1600, 3)

In [30]:
czi.get_dims_shape()  # [{'X': (0, 924), 'Y': (0, 624), 'Z': (0, 1), 'C': (0, 1), 'T': (0, 1), 'M': (0, 2), 'S': (0, 1)}]

[{'A': (0, 3),
  'X': (0, 1600),
  'Y': (0, 1200),
  'C': (0, 1),
  'M': (0, 124),
  'S': (0, 1)}]

In [31]:
czi.is_mosaic()  # True

True

In [32]:
# Mosaic files ignore the S dimension and use an internal mIndex to reconstruct, the scale factor allows one to generate a manageable image
mosaic_data = czi.read_mosaic(C=0, scale_factor=1)
mosaic_data

array([[[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        ...,

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]]], dtype=uint8)

In [33]:
mosaic_data.shape  # (1, 1, 624, 1756)
# the C channel has been specified S & M are used internally for position so this is (T, Z, Y, X)

(1, 18496, 16008, 3)

In [34]:
normed_mosaic_data = norm_by(mosaic_data[0, 0, :, :], 5, 98) * 255
normed_mosaic_data

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [35]:
img = Image.fromarray(normed_mosaic_data.astype(np.uint8))

In [38]:
    tiff_dir = '/Users/mattzabriskie/Desktop/czi_conversion'
    filename = 'czi_files/2023_03_02__0005'
    
    
    # Construct the output file path (change the extension to .tiff)
    tiff_file = os.path.join(tiff_dir, os.path.splitext(filename)[0] + '.tiff')
    
    # Save as a TIFF file
    tifffile.imsave(tiff_file, mosaic_data)

/var/folders/l_/nymzk9_d07q4wpr9pf8wg5ym0000gp/T/ipykernel_52310/3147606794.py:9: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(tiff_file, mosaic_data)


In [ ]:
import os
import czifile
import tifffile
from tqdm import tqdm

# Path to directory containing the CZI files
czi_dir = '/Users/mattzabriskie/Desktop/czi_conversion/czi_files'
tiff_dir = '/Users/mattzabriskie/Desktop/czi_conversion'

# Directory to save the TIFF files
# creates subfolder within directory 'TIFF files'
tiff_dir = os.path.join(tiff_dir, 'tiff_files')
os.makedirs(tiff_dir, exist_ok=True)

# Get all files in directory ending with '.czi'
czi_files = [file for file in os.listdir(czi_dir) if file.endswith('.czi')]


# Loop over all files in the directory
for filename in tqdm(czi_files, desc="Converting files"):
    # Construct the full file path
    czi_file = os.path.join(czi_dir, filename)
    print(filename)

    try:
        # Try to read the CZI file
        with czifile.CziFile(czi_file) as czi:
            img_data = czi.asarray()

    except Exception as e:
        print(f"File {filename} raised an error during processing: {str(e)}. Continuing with next file.")        
        continue
        
    # Construct the output file path (change the extension to .tiff)
    tiff_file = os.path.join(tiff_dir, os.path.splitext(filename)[0] + '.tiff')
    
    # Save as a TIFF file
    tifffile.imsave(tiff_file, img_data)

print("All CZI files have been converted to TIFF.")
